**Google Colab Notebook for the selection of compounds with conjugated $\pi$ systems from PubChemQC Project's B3LYP/6-31G* Dataset**



*   Author: Alfonso Esqueda García (esquedal94@gmail.com)

Descripction:

This script searches for chemical compounds that have certain attributes of interest from a public chemistry database stored in google drive (about 75 million molecules in the entiry database)


For this Google Colab's notebook to work properly a shortcut of PubChemQC Project's b3lyp/6-31G* dataset in your Drive is needed. Follow this link to create said shortcut: 

https://drive.google.com/drive/u/0/folders/1N5q9p-H5TDggFWt8b0c26iwWL6WcGe6i 


-- PubChemQC Project website: http://pubchemqc.riken.jp/

-- PubChemQC Project article: https://pubs.acs.org/doi/10.1021/acs.jcim.7b00083


In [ ]:
# Load PubChemQC Project's B3LYP/6-31G* Dataset's Google Drive Shared Folder
from google.colab import drive
drive.mount('/content/drive/')
PCQCP_dir = '/content/drive/MyDrive/b3lyp_JCIM2017/'

Mounted at /content/drive/


In [ ]:
# Installation & importation of needed libraries
# --Installs
!pip install rdkit-pypi

!pip install -U openbabel-wheel

# --Imports
from os import listdir, mkdir, rmdir, remove
from os.path import isfile, isdir, join

from collections import Counter

import shutil

from multiprocessing import Pool, Lock 

import pandas as pd 

import hashlib

import json

import tarfile # Library needed for the decompressing of .tar.gz files

from rdkit import Chem # Chemical Toolbox Library

     |████████████████████████████████| 22.5 MB 1.4 MB/s 
     |████████████████████████████████| 10.9 MB 26.2 MB/s 


**Function Definitions**

In [ ]:
# Global variables needed

# Get as a list with every file in the Drive's shared folder that contains the
# dataset
ds_files = [f for f in listdir(PCQCP_dir) if isfile(join(PCQCP_dir, f))]
dataset = [f for f in ds_files if '.md5' not in f]  # .md5 file will help to ve- 
                                                    # rify encryption signature
                                                    # to assert the file's 
                                                    # integrity

output_dir = '/content/drive/MyDrive/PubChemQC_Project/'

tmp_outdir = '/content/tmp_PCQCP/'
try:
    mkdir(tmp_outdir)
except FileExistsError:
    pass

cjtd_csv = '/content/drive/MyDrive/PubChemQC_Project/Conjugated_Molecules.csv'

done_dict = output_dir + 'chunks_done.txt'

# Definitions
def iterate_DSChunks(id):
    """
    Iterate the .tar.gz files that, combined, make up the complete B3LYP/6-31G*
    dataset in the search of different chemical properties (Will be used to
    look for molecules with conjugated pi systems)

    int id : an index of the list containing all the .tar.gz files

    return None
    """
    try:
        chunk = dataset[id]
    except IndexError:
        return

    lock1.acquire()
    try:
        with open(done_dict, 'r') as f:
            chunks_done = json.load(f)
    finally:
        lock1.release()

    if chunks_done[chunk]:
        return
    else:
        # Lets verify the encryption signature of the file
        chunk_compressed = PCQCP_dir + chunk
        md5_file = chunk_compressed + '.md5'

        original_md5 = ''
        with open(md5_file, 'r') as f:
            original_md5 = f.readline().split()[0]

        with open(chunk_compressed, 'rb') as file_to_check:
            # read contents of the file
            data = file_to_check.read() 

            # pipe contents of the file through
            md5_returned = hashlib.md5(data).hexdigest()

            # Finally compare original MD5 with freshly calculated
            if original_md5 != md5_returned:
                return

    # Extraction of the folder 
    tar_gz = tarfile.open(chunk_compressed)

    # Directory that contains the extracted files
    chunk_dir = tmp_outdir + chunk.replace('.tar.gz', '/') 

    # Extracting file
    tar_gz.extractall(tmp_outdir)
    tar_gz.close()

    # Get a list with every folder that was extracted
    compounds_dirs = [join(chunk_dir, o) for o in listdir(chunk_dir) 
                      if isdir(join(chunk_dir,o))]

    # Loop for every compound directory
    for compound in compounds_dirs:
        molfiles = [f for f in listdir(compound) if (isfile(join(compound, f))
                    and '.mol' in f)]

        # Loop through every molfile to search for conjugated bonds
        for mol in molfiles:
            mol_fname = join(compound, mol)

            try:
                candidate_mol = Chem.MolFromMolFile(mol_fname)
            except:
                continue

            # Try to generate SMILE from molfile
            try: 
                candidate_smi = Chem.MolToSmiles(candidate_mol)
            except:
                continue

            # Let's feed the SMILE to rdkit 
            rdkit_mol = Chem.MolFromSmiles(candidate_smi)

            for bond in rdkit_mol.GetBonds():
                if bond.GetIsConjugated():
                    lock2.acquire() # Lock to prevent data overwrite
                    try:
                        with open(cjtd_csv, 'a') as f:
                            f.write('\n')
                            nline = str(candidate_smi) + ', ' + str(mol_fname)
                            f.write(nline)
                    finally:
                        lock2.release()
                        
                    # Copy molfile to output directory
                    #dst_name = molfiles_dir + mol
                    #shutil.copyfile(mol_fname, dst_name)

                    # Get out of bond iterating loop
                    break
                else: 
                    continue
         
        # Compound's bonds search ends here, let's erase the compound dir
        try:
            shutil.rmtree(compound) # remove compound dir
        except shutil.Error:
            pass

    # When every compound is searched erase the tmp_folder created
    try:
        shutil.rmtree(chunk_dir) # remove chunk dir
    except shutil.Error:
        pass
    
    lock1.acquire()
    try:
        with open(done_dict, 'r') as f:
            chunks_done = json.load(f)
        
        chunks_done[chunk] = True
        
        with open(done_dict, 'w') as f:
            json.dump(chunks_done, f)
    finally:
        lock1.release()

    lock3.acquire()
    try:
        logfile = output_dir + 'progress.log'

        with open(logfile, 'w') as f:
            done_ctr = Counter(chunks_done.values())[True]
            nof_chunks = len(chunks_done)
            buff = '\r Progreso: ' + str(done_ctr) + ' chunks analizados de '
            buff += str(nof_chunks) + ' ----- '  
            buff += str(round(((float(done_ctr)/nof_chunks) * 100), 1)) + '%'
            f.write(buff)
    finally:
        lock3.release()

**---------------------------------------------**

**- Run the iterate_DSChunks function with multiple parallel threads**

In [ ]:
def init(l1, l2, l3):
    global lock1
    global lock2
    global lock3
    
    lock1 = l1
    lock2 = l2
    lock3 = l3

if __name__ == '__main__':
    iter = [i for i in range(0, len(dataset))]

    l1 = Lock()
    l2 = Lock()
    l3 = Lock()

    with Pool(7, initializer=init, initargs=(l1, l2, l3,)) as p:
        p.map(iterate_DSChunks, iter)
        p.close()
        p.join()

In [ ]:
EXECUTE = False

In [ ]:
# Before iterating through every compound in the dataset we need to populate a  
# dictionary with values 'done' or 'not done' (True or False) for practicity for 
# every chunk in the dataset folder 

# THE FOLLOWING CODE ONLY NEEDS TO BE RUN ONCE. Its purpose is to create for the
# first time the done_dict file (the value for every key will be False ('not
# done))
if EXECUTE:
    fresh_dict = {}

    for chunk in dataset:
        fresh_dict[chunk] = False

    with open(done_dict, 'w') as f:    
        json.dump(fresh_dict, f)

In [ ]:
# MANUALLY erase the content of the tmp_PCQCP folder 
if EXECUTE:
    !rm -rf /content/tmp_PCQCP/* # This might only work in Google Colab

In [ ]:
# MANUALLY erase the content of the Conjugated Molecules CSV

if EXECUTE:
    # This might only work in Google Colab
    !rm  /content/drive/MyDrive/PubChemQC_Project/Conjugated_Molecules.csv

    with open(cjtd_csv, 'w') as f:
        f.write('SMILE, PATH')  